In [69]:
from numpy import array, zeros_like, concatenate, logspace, identity, sum, log as ln, empty
from numpy.random import randint, seed
from numpy.linalg import cond, inv
from scipy.linalg import toeplitz

N_u = 500
seed(31)
u = randint(0,2,size=N_u)

y = zeros_like(u, dtype='float64')
# ARX(n, m) = ARX(2,1) model
# y = array([.4*y[n-1] - .6*y[n-2] + 2*u[n-1] for n in range(2, N_u)])
for n in range(2, N_u):
    y[n] = .4*y[n-1] - .6*y[n-2] + 2*u[n-1]

def my_var(x):
    N = len(x)
    u_x = sum(x)/N
    return sum((x-u_x)**2)/(N-1)

# Question 1

In [71]:
# The first half of the generated signal
N = N_u // 2
AIC_p, BIC_p = empty(3), empty(3)
all_n, all_m = array((1, 2, 2)), array((1, 1, 2))

for i, (n, m) in enumerate(zip(all_n, all_m)):
    q = max(n,m) + 1
    # transformation matrix -> [N-q+1 x n+m] -> [N-max(n,m) x n+m]
    # OBS: Due to Python has a zero-based indexing, y[k-1]-> y(k)
    X = concatenate((toeplitz(y[q-2:N-1], y[q-n-1:q-1][::-1]), toeplitz(u[q-2:N-1], u[q-m-1:q-1][::-1])), axis=1)
    # Tikhonov regularization
    for lambda_ in logspace(0,10,base=2):
        if cond(X.T@X + (lambda_-1)*identity(X.shape[1])) < 1e3:
            # Ordinary Least-Square (OLS)
            theta_hat = inv(X.T@X + (lambda_-1)*identity(n+m))@X.T@y[q-1:N]
            break
    # estimated signal
    y_hat = X @ theta_hat
    # residue
    e = y[q-1:N] - y_hat
    N_e = e.size
    sigma2_e = my_var(e)
    p = m+n
    # Akaike’s Information criterion (AIC)
    AIC_p[i] = N_e*ln(sigma2_e) + 2*p
    # Bayesian Information criterion (BIC)
    BIC_p[i] = N_e*ln(sigma2_e) + p*ln(N_e)

print("Akaike’s Information criterion (AIC):")
for i, n, m in zip(range(AIC_p.size), all_n, all_m):
    print(f'\tAIC({n},{m}) = {AIC_p[i]:.2e}')
print(f'\tThe best option for the AIC is {{n, m}}={{{all_n[AIC_p == min(AIC_p)][0]}, {all_m[AIC_p == min(AIC_p)][0]}}}\n\n')

print('Bayesian Information criterion (BIC):')
for i, n, m in zip(range(AIC_p.size), (1, 2, 2), (1, 1, 2)):
    print(f'\tBIC({n},{m}) = {BIC_p[i]:.2e}')
print(f'\tThe best option for the BIC is {{n, m}}={{{all_n[BIC_p == min(BIC_p)][0]}, {all_m[BIC_p == min(BIC_p)][0]}}}')

Akaike’s Information criterion (AIC):
	AIC(1,1) = -1.57e+02
	AIC(2,1) = -1.74e+04
	AIC(2,2) = -1.72e+04
	The best option for the AIC is {n, m}={2, 1}


Bayesian Information criterion (BIC):
	BIC(1,1) = -1.50e+02
	BIC(2,1) = -1.73e+04
	BIC(2,2) = -1.72e+04
	The best option for the BIC is {n, m}={2, 1}
